In [1]:
!pip install nltk


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aaryanpotdar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from tqdm.auto import tqdm
import nltk
from nltk.tokenize import sent_tokenize

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df_2022 = pd.read_csv('/Users/aaryanpotdar/Desktop/Kaggle_CLEF/csv_files/data_2022.csv')

In [5]:
df_2022

,DATE,TEXT,USER
0,2021-11-21 09:04:18,i am so in love. thats it thats the post i am ...,eRisk2022-T3_Subject1
1,2021-11-21 08:32:52,me but im a guy lookin at skinny guys lmfao,eRisk2022-T3_Subject1
2,2021-11-16 09:11:26,"anyone else just say ""fuck you"" to one particu...",eRisk2022-T3_Subject1
3,2021-11-15 11:51:31,"im really sorry youre goin through this man, i...",eRisk2022-T3_Subject1
4,2021-11-15 11:47:12,is there anyone around you right now you could...,eRisk2022-T3_Subject1
...,...,...,...
8472,2020-10-16 00:17:39,should i watch past season 3?. this is the fir...,eRisk2022-T3_Subject28
8473,2020-10-07 21:17:22,hagrid's other sidequest. is there a list of a...,eRisk2022-T3_Subject28
8474,2020-10-04 01:40:27,peeta revived. ever since reading the books an...,eRisk2022-T3_Subject28
8475,2020-09-26 00:50:33,itap: a sunset through a window.,eRisk2022-T3_Subject28


In [4]:
# BERT Tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [5]:
if torch.cuda.is_available():
    model.cuda()

In [8]:
# ensure strings are not empty and not NaN
df_2022['TEXT'] = df_2022['TEXT'].fillna('').astype(str)
df_2022['TEXT'] = df_2022['TEXT'].apply(lambda x: x if x.strip() != '' else 'emptytext')

In [9]:
df_2022

,DATE,TEXT,USER
0,2021-11-21 09:04:18,i am so in love. thats it thats the post i am ...,eRisk2022-T3_Subject1
1,2021-11-21 08:32:52,me but im a guy lookin at skinny guys lmfao,eRisk2022-T3_Subject1
2,2021-11-16 09:11:26,"anyone else just say ""fuck you"" to one particu...",eRisk2022-T3_Subject1
3,2021-11-15 11:51:31,"im really sorry youre goin through this man, i...",eRisk2022-T3_Subject1
4,2021-11-15 11:47:12,is there anyone around you right now you could...,eRisk2022-T3_Subject1
...,...,...,...
8472,2020-10-16 00:17:39,should i watch past season 3?. this is the fir...,eRisk2022-T3_Subject28
8473,2020-10-07 21:17:22,hagrid's other sidequest. is there a list of a...,eRisk2022-T3_Subject28
8474,2020-10-04 01:40:27,peeta revived. ever since reading the books an...,eRisk2022-T3_Subject28
8475,2020-09-26 00:50:33,itap: a sunset through a window.,eRisk2022-T3_Subject28


In [10]:
#running sentence tokenizer to find max sentence length.
max_sentence_length = 0
for text in df_2022['TEXT']:
    sentences = sent_tokenize(text)
    for sentence in sentences:
        
        words = nltk.word_tokenize(sentence)
        max_sentence_length = max(max_sentence_length, len(words))

print(f"max_length: {max_sentence_length}")

max_length: 177


In [11]:
#max length is 177
# 177 < 512 so bert can handle it

In [12]:
!pip install tqdm
# for adding progress bar


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [6]:
def split_and_pool(text, tokenizer, model, max_length=512):
    
    sentences = sent_tokenize(text)

    pooled_embeddings = []

    for sentence in tqdm(sentences, desc='Processing sentences', leave=False):
        tokens = tokenizer.tokenize(sentence)

        for chunk_tokens in np.array_split(tokens, np.ceil(len(tokens) / (max_length - 2))):
            chunk = ['[CLS]'] + chunk_tokens.tolist() + ['[SEP]']
            input_ids = tokenizer.convert_tokens_to_ids(chunk)
            input_ids_tensor = torch.tensor([input_ids])

            if torch.cuda.is_available():
                input_ids_tensor = input_ids_tensor.cuda()

            with torch.no_grad():
                outputs = model(input_ids_tensor)

            cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()

            pooled_embeddings.append(cls_embedding)
            
    final_embedding = np.mean(np.vstack(pooled_embeddings), axis=0)
    return final_embedding

In [14]:
embeddings_2022 = []
for text in tqdm(df_2022['TEXT'].tolist(), desc='Overall Progress'):
    embedding = split_and_pool(text, tokenizer, model)
    embeddings_2022.append(embedding)

Overall Progress: 100%|██████████| 8477/8477 [21:11<00:00,  6.67it/s]   


In [17]:
# convert embeddings into np_array
embeddings_2022 = np.array(embeddings_2022)

In [18]:
embeddings_2022.shape

(8477, 768)

In [19]:
for i in range(embeddings_2022.shape[1]):
    df_2022[f'embedding_{i}'] = embeddings_2022[:, i]

/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_82679/1312096501.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2022[f'embedding_{i}'] = embeddings_2022[:, i]
/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_82679/1312096501.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2022[f'embedding_{i}'] = embeddings_2022[:, i]
/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_82679/1312096501.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

In [21]:
df_2022.drop(columns=['DATE'], inplace=True)
df_2022.drop(columns=['TEXT'], inplace=True)

In [22]:
df_2022

,USER,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,eRisk2022-T3_Subject1,-0.022457,0.336692,0.099213,-0.120019,-0.321374,-0.485824,0.346999,0.706476,0.406262,...,0.045692,-0.469622,0.234385,-0.482862,0.230412,-0.077851,-0.329253,-0.484919,0.569902,0.321189
1,eRisk2022-T3_Subject1,-0.259201,0.379615,-0.042923,-0.316316,-0.560104,-0.016254,0.525716,0.484025,0.167928,...,0.078041,-0.622565,0.219295,-0.046370,-0.054232,0.433260,-0.271939,-0.493637,0.608423,0.266888
2,eRisk2022-T3_Subject1,0.220104,0.057617,-0.305687,0.051585,-0.633367,-0.311230,0.295637,0.566354,-0.182671,...,0.116304,0.005103,0.626910,0.203249,0.004213,0.362805,-0.471337,-0.151651,0.239074,0.461023
3,eRisk2022-T3_Subject1,-0.261329,0.002115,-0.035605,-0.212768,-0.522489,-0.363408,0.704951,0.567608,0.229827,...,-0.011730,-0.220560,0.327083,0.156205,0.214784,0.346514,-0.208380,-0.271254,0.461150,0.277571
4,eRisk2022-T3_Subject1,-0.236614,0.013746,-0.312270,-0.241477,-0.223823,-0.252511,0.532251,0.676057,0.182644,...,0.175187,-0.391872,0.280690,-0.225986,0.030410,0.183673,-0.283554,-0.570620,0.144529,0.438334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8472,eRisk2022-T3_Subject28,0.057581,-0.232186,0.144007,-0.175823,-0.150277,-0.445756,0.234716,0.539154,0.043737,...,-0.205907,-0.304751,0.035648,-0.216306,0.141817,-0.066460,-0.157521,-0.224355,0.502873,0.244605
8473,eRisk2022-T3_Subject28,-0.315856,-0.179040,-0.024314,0.010473,-0.366583,-0.043597,0.017935,0.305904,-0.214497,...,-0.114697,0.029956,0.214864,-0.040879,0.031303,-0.173753,-0.314102,-0.032681,0.383309,0.425690
8474,eRisk2022-T3_Subject28,-0.114693,0.020083,-0.264603,0.002111,-0.382927,-0.214443,0.213774,0.279294,0.067240,...,-0.062615,0.002711,0.154730,-0.099194,-0.015727,0.103830,-0.160877,-0.149036,0.329812,0.616426
8475,eRisk2022-T3_Subject28,-0.358951,-0.278134,-0.116053,0.104734,-0.554830,-0.183788,0.003241,0.598302,-0.067960,...,0.181495,-0.094871,-0.183681,0.172212,0.009120,0.371086,-0.103648,-0.270480,0.277842,0.530757


In [23]:
df_2022.to_csv('csv_files/emb_2022.csv')

In [24]:
df_2023 = pd.read_csv('/Users/aaryanpotdar/Desktop/Kaggle_CLEF/csv_files/data_2023.csv')

df_2023['TEXT'] = df_2023['TEXT'].fillna('').astype(str)
df_2023['TEXT'] = df_2023['TEXT'].apply(lambda x: x if x.strip() != '' else 'emptytext')

df_2023

,DATE,TEXT,USER
0,2022-10-18 16:54:09,"idk mate, since 1796 we have been using vaccin...",eRisk2022-T3_Subject1
1,2022-10-18 09:18:57,no disrespect intended i'm surennhow the actua...,eRisk2022-T3_Subject1
2,2022-10-18 09:12:08,the older kids would and the baby would see th...,eRisk2022-T3_Subject1
3,2022-10-18 09:11:46,no,eRisk2022-T3_Subject1
4,2022-10-18 09:11:23,it's also used when the head is deformed becau...,eRisk2022-T3_Subject1
...,...,...,...
15128,2018-08-15 21:19:55,had an ex literally tell me he would try not t...,eRisk2022-T3_Subject46
15129,2018-08-15 21:09:54,my boyfriend wanted to see if he could stick t...,eRisk2022-T3_Subject46
15130,2018-08-15 20:48:30,turkey in german translates to threat chicken.,eRisk2022-T3_Subject46
15131,2018-08-15 20:38:25,i had written a suicide note in case i ever wa...,eRisk2022-T3_Subject46


In [25]:
#running sentence tokenizer to find max sentence length.
max_sentence_length = 0
for text in df_2023['TEXT']:
    sentences = sent_tokenize(text)
    for sentence in sentences:
        
        words = nltk.word_tokenize(sentence)
        max_sentence_length = max(max_sentence_length, len(words))

print(f"max_length: {max_sentence_length}")

max_length: 499


In [ ]:
# max len for 2023 dataset is 499. 499 < 512 - 2. So no change to code. Sent tokenization is enough.

In [26]:
embeddings_2023 = []
for text in tqdm(df_2023['TEXT'].tolist(), desc='Overall Progress'):
    embedding = split_and_pool(text, tokenizer, model)
    embeddings_2023.append(embedding)

Overall Progress: 100%|██████████| 15133/15133 [24:39<00:00, 10.23it/s] 


In [27]:
embeddings_2023 = np.array(embeddings_2023)

In [28]:
for i in range(embeddings_2023.shape[1]):
    df_2023[f'embedding_{i}'] = embeddings_2023[:, i]

/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_82679/2697002948.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2023[f'embedding_{i}'] = embeddings_2023[:, i]
/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_82679/2697002948.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2023[f'embedding_{i}'] = embeddings_2023[:, i]
/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_82679/2697002948.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

In [30]:
df_2023.drop(columns=['DATE'], inplace=True)
df_2023.drop(columns=['TEXT'], inplace=True)

In [31]:
df_2023.to_csv('csv_files/emb_2023.csv')

In [32]:
df_2023

,USER,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,eRisk2022-T3_Subject1,0.307920,0.184283,0.196719,-0.378851,-0.775362,-0.655857,0.702414,1.133344,-0.005397,...,-0.495313,-0.339461,-0.063048,-0.661524,0.327759,0.040376,-0.700129,-0.778192,0.915712,0.398367
1,eRisk2022-T3_Subject1,0.461805,0.358137,-0.178235,-0.231508,-0.422522,-0.413948,0.502760,0.761817,0.147534,...,0.011284,-0.417405,0.203851,-0.040265,0.184391,0.235443,-0.239710,-0.195845,0.372403,0.680309
2,eRisk2022-T3_Subject1,0.111218,0.006792,0.554233,0.438290,-0.282072,-0.111621,0.024060,0.721943,-0.307065,...,-0.237357,0.067343,0.043203,0.087566,-0.017523,0.109097,-0.617742,-0.581974,0.597942,0.203909
3,eRisk2022-T3_Subject1,-0.268096,0.206103,-0.136456,-0.147714,-0.248952,-0.100665,0.234014,0.704071,-0.227196,...,0.353186,0.083220,0.074159,0.064315,-0.001543,-0.155215,-0.058098,-0.168255,0.391619,0.448321
4,eRisk2022-T3_Subject1,-0.070742,-0.077476,-0.167748,-0.222121,-0.249834,-0.067934,0.246648,0.770311,0.101785,...,-0.424063,-0.229650,0.258770,-0.221384,0.169118,0.134783,-0.356719,-0.203541,0.065835,0.717534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15128,eRisk2022-T3_Subject46,0.042898,0.003495,-0.149577,0.088537,-0.390601,-0.165491,0.037180,0.391094,0.173257,...,0.240314,-0.085689,0.210298,-0.105496,0.125298,-0.011269,-0.259373,-0.261037,0.278844,0.426380
15129,eRisk2022-T3_Subject46,0.035937,-0.033237,-0.205342,0.041820,-0.191794,-0.184553,0.215643,0.541781,0.221762,...,-0.289904,-0.314036,0.288133,-0.219249,0.207328,0.058100,-0.210333,-0.226979,0.370330,0.390766
15130,eRisk2022-T3_Subject46,-0.547722,0.248518,-0.476352,-0.524485,-0.436975,-0.047998,0.388187,0.908541,-0.460302,...,0.185672,-0.173443,0.288274,-0.209897,0.415613,0.306977,0.127434,-0.595985,0.935346,0.681289
15131,eRisk2022-T3_Subject46,0.083540,0.020122,-0.052048,-0.178230,-0.431820,-0.182695,0.112793,0.626425,0.284546,...,0.107758,-0.103112,0.307847,-0.237421,0.087465,0.067560,-0.431473,0.066942,0.178756,0.372505


In [8]:
df_test = pd.read_csv('/Users/aaryanpotdar/Desktop/Kaggle_CLEF/csv_files/test_data.csv')

df_test['TEXT'] = df_test['TEXT'].fillna('').astype(str)
df_test['TEXT'] = df_test['TEXT'].apply(lambda x: x if x.strip() != '' else 'emptytext')

df_test

,DATE,TEXT,USER
0,2024-01-09 11:01:12,more consistent exposure to your hand will pro...,Subject1
1,2023-11-26 08:56:37,replacing the substrate is probably for the be...,Subject1
2,2023-10-30 01:05:19,"sprite has more prominent banding though, hes ...",Subject1
3,2023-10-30 01:04:35,amp;formatpjpg amp;autowebp amp;s61891ea1313c...,Subject1
4,2023-10-14 05:47:09,"below my goal weight, now the goal has just sh...",Subject1
...,...,...,...
7514,2023-10-14 02:08:28,"when i beta for someone, i also start off with...",Subject18
7515,2023-10-13 19:54:59,i don't think i ever truly knew you,Subject18
7516,2023-10-13 12:28:59,"there's eminence in shadow, although it's more...",Subject18
7517,2023-10-13 01:44:05,i wouldn't be offended at all. the only thing ...,Subject18


In [9]:
#running sentence tokenizer to find max sentence length.
max_sentence_length = 0
for text in df_test['TEXT']:
    sentences = sent_tokenize(text)
    for sentence in sentences:
        
        words = nltk.word_tokenize(sentence)
        max_sentence_length = max(max_sentence_length, len(words))

print(f"max_length: {max_sentence_length}")

max_length: 309


In [ ]:
# max len for testing dataset is 309. 309 < 512 - 2. So no change to code. Sent tokenization is enough.

In [10]:
embeddings_test = []
for text in tqdm(df_test['TEXT'].tolist(), desc='Overall Progress'):
    embedding = split_and_pool(text, tokenizer, model)
    embeddings_test.append(embedding)

Overall Progress: 100%|██████████| 7519/7519 [18:22<00:00,  6.82it/s]   


In [11]:
embeddings_test = np.array(embeddings_test)

In [12]:
for i in range(embeddings_test.shape[1]):
    df_test[f'embedding_{i}'] = embeddings_test[:, i]

/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_11363/1249835472.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[f'embedding_{i}'] = embeddings_test[:, i]
/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_11363/1249835472.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[f'embedding_{i}'] = embeddings_test[:, i]
/var/folders/g_/q1mhhlqx2kvc2wsjsrmc7kx00000gn/T/ipykernel_11363/1249835472.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

In [13]:
df_test.drop(columns=['DATE'], inplace=True)
df_test.drop(columns=['TEXT'], inplace=True)

In [ ]:
df_test.to_csv('csv_files/emb_test.csv')

In [14]:
df_test

,USER,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,Subject1,-0.072027,0.117544,-0.024513,-0.362428,-0.537908,-0.433939,0.091954,0.537988,0.770397,...,-0.121276,-0.102062,0.232782,-0.286756,0.171106,0.321854,-0.225586,-0.273288,0.525084,0.492826
1,Subject1,-0.173974,0.194406,-0.208257,0.054306,-0.203133,-0.498073,0.240525,0.719508,0.122839,...,-0.135093,-0.344488,0.206339,0.077439,0.011982,0.288609,-0.190208,-0.127556,0.178047,0.497107
2,Subject1,-0.432588,-0.410271,-0.491965,-0.175570,-0.411142,-0.249240,0.393846,0.389656,0.208259,...,-0.244405,0.082435,0.137110,-0.047537,0.368410,0.279262,-0.004822,-0.201456,1.117711,0.285813
3,Subject1,-0.156224,-0.251372,0.015289,0.006797,-0.398336,0.035703,0.040379,0.283533,-0.103094,...,0.007751,-0.208741,0.214994,0.251831,0.502341,0.192172,-0.196032,0.023102,0.136884,0.463758
4,Subject1,-0.305068,-0.026600,0.030862,0.006466,-0.267850,-0.071505,0.352290,0.420227,-0.042371,...,0.310831,0.091647,0.160889,0.018344,0.187901,0.030385,0.136087,-0.376439,0.096566,0.565359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7514,Subject18,0.125881,0.261106,-0.252636,-0.368598,-0.386131,-0.219428,0.211567,0.562050,0.264253,...,-0.338463,-0.307492,0.073405,-0.200391,0.000701,0.100048,-0.051348,-0.125357,0.164694,0.660239
7515,Subject18,0.172623,0.136061,-0.119321,0.035150,-0.284074,-0.361906,0.427074,0.597285,-0.162653,...,0.128674,-0.594550,0.247146,-0.028827,0.249608,0.074043,0.065057,-0.079368,0.208377,0.276980
7516,Subject18,0.082023,-0.160851,-0.193370,-0.021431,-0.547860,-0.349370,0.053164,0.462721,0.185631,...,-0.257056,-0.334448,-0.137110,0.044021,-0.106367,0.059120,-0.143163,-0.007731,0.429831,0.531334
7517,Subject18,0.244429,0.056481,-0.286747,-0.096453,-0.463076,-0.334498,0.207813,0.540986,0.136466,...,0.072153,-0.227923,0.101447,-0.068244,-0.053858,0.042077,-0.280002,-0.125584,0.176294,0.442782
